In [12]:
import sys
import itertools
from anytree import Node, RenderTree,Resolver,ChildResolverError,NodeMixin,PreOrderIter
from anytree.dotexport import RenderTreeGraph

class CustomNode(NodeMixin):  # Add state number feature
    def __init__(self, name, state_number,symbol,end_state, parent=None):
        super(CustomNode, self).__init__()
        self.name=name
        self.state_number=state_number
        self.parent=parent
        self.end_state=end_state
        self.symbol=symbol

state_number=0
top=CustomNode("root",state_number,'',False)
r = Resolver('name')




In [13]:
def add_inflectional_suffixes(suffix,state_number,root_node):
    suffixes=suffix.split(":")
    state_number+=1
    
    node=CustomNode(suffixes[0],state_number,suffixes[1],True,root_node)
    if suffixes[1]=='+N':
        state_number+=1
        plural_node=CustomNode('^s',state_number,'+PL',True,node)
        state_number+=1
        CustomNode("^s",state_number,'+SGPS',True,node)
        state_number+=1
        CustomNode("'",state_number,'+PLPS',True,plural_node)
    elif suffixes[1]=='+V':
        state_number+=1
        CustomNode("^s",state_number,'+V+3D',True,node)
        state_number+=1
        CustomNode("ing",state_number,'+ING',True,node)
    return (node,state_number)
def append_suffixes(root_node,morphs,index,state_number):
    suffix=''
    while len(morphs)>index:
        index+=1
        if morphs[index]=='|':
            if suffix!='': #check if suffix to be handle before pipe
                
                node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
                suffix=''
        elif morphs[index]=='(':
            node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
            index,state_number=append_suffixes(node,morphs,index,state_number)
            suffix=''
        elif morphs[index]==')':
            if suffix!='': #check if suffix to be handle before paranthesis
                val=add_inflectional_suffixes(suffix,state_number,root_node)
                state_number=val[1]
            return (index,state_number)
        else:
            suffix+=morphs[index]
    return (index,state_number)

def insert_irregular_noun(stem,state_number,root):
    verbs=stems=stem.split('/')
    root_stem=verbs[0]
    for i in range(len(root_stem)):
        state_number+=1
        root=CustomNode(stem[i],state_number,'<eps>',False,root)
        
    state_number+=1
    n=CustomNode(verbs[0],state_number,verbs[0]+'+N+SG',True,root)
    
    state_number+=1
    n=CustomNode(verbs[1],state_number,verbs[1]+'+N+PL',True,root)
            
    return (root,state_number)

In [14]:
#words = ['compute (er:+N(ize:+V(ation:+N|able:+ADJ)|less:+ADJ(ness:+N))|ation:+N(able:+ADJ))','work (er:+N(ism:+N))']
#words = ["be/were&was/been (ing:+N)","begin/began/begun (ing:+N|er:+N)"]
with open('Lexicon/Nouns.txt') as f:
    words = f.readlines()
    words.sort()
    for word in words:
        line=word.rstrip()
        arr=line.split(' ')
        stem=arr[0]
        if len(arr)==2:
            morphs=arr[1]
        root=top

        if '/' in stem: #means irregular verb
            root,state_number=insert_irregular_noun(stem,state_number,top)
        else:
            for i in range(len(stem)):
                try:
                    n=r.get(root,stem[i])
                except ChildResolverError:
                    state_number+=1
                    n=CustomNode(stem[i],state_number,stem[i],False,root)
                root=n
            state_number+=1
            CustomNode("<eps>",state_number,'+N+SG',True,root)
            state_number+=1
            CustomNode("^s",state_number,'+N+PL',True,root)
        
        if len(arr)==2:
            return_val=append_suffixes(root,morphs,0,state_number)
            state_number=return_val[1]
RenderTreeGraph(top).to_picture("udo.png")

In [15]:
p=PreOrderIter(top)

p=itertools.islice(p,1,None) #pass root


for n in p:
    print(n.name + " " +str(n.state_number) + "parent: " + n.parent.name)

a 1parent: root
r 2parent: a
t 3parent: r
<eps> 4parent: t
^s 5parent: t
d 6parent: root
a 7parent: d
t 8parent: a
a 9parent: t
<eps> 10parent: a
^s 11parent: a
h 12parent: root
e 13parent: h
a 14parent: e
l 15parent: a
t 16parent: l
h 17parent: t
<eps> 18parent: h
^s 19parent: h
i 20parent: h
s 21parent: i
t 22parent: s
o 23parent: t
r 24parent: o
y 25parent: r
<eps> 26parent: y
^s 27parent: y
m 28parent: root
e 29parent: m
a 30parent: e
t 31parent: a
<eps> 32parent: t
^s 33parent: t
t 34parent: e
h 35parent: t
o 36parent: h
d 37parent: o
<eps> 38parent: d
^s 39parent: d
u 40parent: m
s 41parent: u
i 42parent: s
c 43parent: i
<eps> 44parent: c
^s 45parent: c
n 46parent: root
o 47parent: n
u 48parent: o
n 49parent: u
<eps> 50parent: n
^s 51parent: n
p 52parent: root
e 53parent: p
r 54parent: e
s 55parent: r
o 56parent: s
n 57parent: o
person 58parent: n
people 59parent: n
s 60parent: root
y 61parent: s
s 62parent: y
t 63parent: s
e 64parent: t
m 65parent: e
<eps> 66parent: m
^s 67paren

In [16]:

#for pre, _, node in RenderTree(top):
#    treestr = u"%s%s" % (pre, node.name)
#    print(treestr.ljust(8), node.state_number)
def print_as_tabs(prev_state,next_state,input_label,output_label):
    print(str(prev_state) + "\t" + str(next_state) + "\t"+ str(input_label) + "\t" + str(output_label))


# In[222]:


p=PreOrderIter(top)
p=itertools.islice(p,1,None) #pass root

print_as_tabs(0,1,"<s>","<s>")
print_as_tabs(1,0,"<eps>","<eps>")
print_as_tabs(0,2,"</s>","</s>")
print_as_tabs(2,0,"<eps>","<eps>")
print_as_tabs(0,3,"<unk>","<unk>")
print_as_tabs(3,0,"<eps>","<eps>")
for n in p:
    parent_state_number=n.parent.state_number
    if n.parent.state_number!=0:
        parent_state_number+=3
    print_as_tabs(parent_state_number,n.state_number+3,n.symbol,n.name)
    if n.end_state:
        print_as_tabs(n.state_number+3,0,"<eps>","#")  
    #if n.is_leaf:
     #   print_as_tabs(n.state_number+3,0,"<eps>","#")
print("0")


0	1	<s>	<s>
1	0	<eps>	<eps>
0	2	</s>	</s>
2	0	<eps>	<eps>
0	3	<unk>	<unk>
3	0	<eps>	<eps>
0	4	a	a
4	5	r	r
5	6	t	t
6	7	+N+SG	<eps>
7	0	<eps>	#
6	8	+N+PL	^s
8	0	<eps>	#
0	9	d	d
9	10	a	a
10	11	t	t
11	12	a	a
12	13	+N+SG	<eps>
13	0	<eps>	#
12	14	+N+PL	^s
14	0	<eps>	#
0	15	h	h
15	16	e	e
16	17	a	a
17	18	l	l
18	19	t	t
19	20	h	h
20	21	+N+SG	<eps>
21	0	<eps>	#
20	22	+N+PL	^s
22	0	<eps>	#
15	23	i	i
23	24	s	s
24	25	t	t
25	26	o	o
26	27	r	r
27	28	y	y
28	29	+N+SG	<eps>
29	0	<eps>	#
28	30	+N+PL	^s
30	0	<eps>	#
0	31	m	m
31	32	e	e
32	33	a	a
33	34	t	t
34	35	+N+SG	<eps>
35	0	<eps>	#
34	36	+N+PL	^s
36	0	<eps>	#
32	37	t	t
37	38	h	h
38	39	o	o
39	40	d	d
40	41	+N+SG	<eps>
41	0	<eps>	#
40	42	+N+PL	^s
42	0	<eps>	#
31	43	u	u
43	44	s	s
44	45	i	i
45	46	c	c
46	47	+N+SG	<eps>
47	0	<eps>	#
46	48	+N+PL	^s
48	0	<eps>	#
0	49	n	n
49	50	o	o
50	51	u	u
51	52	n	n
52	53	+N+SG	<eps>
53	0	<eps>	#
52	54	+N+PL	^s
54	0	<eps>	#
0	55	<eps>	p
55	56	<eps>	e
56	57	<eps>	r
57	58	<eps>	s
58	59	<eps>	o
59	60	<eps>	n
60	61	person+N+SG	perso

In [30]:
def append_suffixes(node_stack,morphs,index,paranthesis_stack,state_number):
    suffix=''
    while len(paranthesis_stack)!=0:
        index+=1
        
        if morphs[index]=='|':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
                suffix=''
        elif morphs[index]=='(':
            suffixes=suffix.split(':')
            state_number+=1
            
            node=CustomNode(suffixes[0],state_number,suffixes[1],node_stack[-1])
            node_stack.append(node)
            paranthesis_stack.append(index)
            param_node_stack=[node_stack[-1]]
            param_paranthesis_stack=[index]
            print(node_stack[-1].name)
            print(param_paranthesis_stack)
        
            var=append_suffixes(param_node_stack,morphs,index,param_paranthesis_stack,state_number)
            index=var[0]
            state_number=var[1]
            node_stack.pop()
            paranthesis_stack.pop()
            suffix=''
            print(node_stack[-1].name)
        elif morphs[index]==')':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
            paranthesis_stack.pop()
            node_stack.pop()
            return (index,state_number,node_stack,paranthesis_stack)
        else:
            suffix+=morphs[index]
    return (index,state_number,node_stack,paranthesis_stack)